# annotate family relations
* train a model for family relation types

In [1]:
import ast
import random
from spacytei.train import batch_train

In [2]:
persons = Person.objects.filter(is_related_person__isnull=False)
persons.count()

9791

In [ ]:
items = [x.written_name for x in persons]
items = sorted(iter(items), key=lambda k: random.random())
filename = "person__related.txt"
with open(filename, 'w', encoding="utf-8") as f:
    for x in items:
        f.write("{}".format(x) + '\n')

In [3]:
file = "./training_data/invs_fam.jsonl"

In [5]:
with open(file) as f:
    TRAIN_DATA = f.readlines()
train_data = [ast.literal_eval(x) for x in TRAIN_DATA]
print(len(train_data))
train_data[4]

350


['Agnes Meixnerin (Tochter der Verstorbenen)', {'entities': [[17, 24, 'FAM']]}]

In [6]:
train_data[4][0][17:24]

'Tochter'

In [14]:
batch_train(train_data=train_data, output_dir='./data/main_fam', new_label='FAM', eval_split=0.2, n_iter=8)

Created blank model from 'de'
280 train vs 70 test samples
Iteration Number: 0
Losses {'ner': 57.807576010540956}
Duration: 0:00:12.615114
p: 93.75; f: 76.92307692307693; r: 65.21739130434783
Saved model to: data\main_fam
######################
Iteration Number: 1
Losses {'ner': 40.95790779389665}
Duration: 0:00:12.413391
p: 87.17948717948718; f: 80.0; r: 73.91304347826086
Saved model to: data\main_fam
######################
Iteration Number: 2
Losses {'ner': 39.7699765055019}
Duration: 0:00:12.777250
p: 87.8048780487805; f: 82.75862068965519; r: 78.26086956521739
Saved model to: data\main_fam
######################
Iteration Number: 3
Losses {'ner': 37.88923923011385}
Duration: 0:00:12.839645
p: 82.0; f: 85.41666666666666; r: 89.13043478260869
Saved model to: data\main_fam
######################
Iteration Number: 4
Losses {'ner': 34.73160183547021}
Duration: 0:00:12.708678
p: 94.87179487179486; f: 87.05882352941177; r: 80.43478260869566
Saved model to: data\main_fam
##################

In [15]:
samples = [
    "Maria Hueberin (Tochter des Michael Hueber und der Ursula Mayrin; Nichte des Verstorbenen)",
    "Veit [Vitus] Schiferegger [Schifferegger] (Sohn des Verstorbenen)",
    "Johann Kofler [Hans Kofler/Joseph Kofler?] (Tischlermeister), Bruneck, (Kurator des Andre Papprian, des Johann Papprian [Hans Papprian], der Maria Papprianin und der Magdalena Papprianin)",
    "Maria Göllerin (abwesende Tochter der Maria Aichnerin, einer Halbschwester des Verstorbenen)",
]

In [16]:
import spacy

In [17]:
nlp = spacy.load(r"./data/main_fam")

In [18]:
for x in samples:
    doc = nlp(x)
    ents = doc.ents
    for ent in ents:
        print(ent)

Nichte
Sohn


In [ ]:
fam_scheme, _ = SkosCollection.objects.get_or_create(name="Familiäre Beziehungen")

In [ ]:
for x in persons:
    doc = nlp(x.written_name)
    ents = doc.ents
    if len(ents) == 1:
        fam_rel_type = None
        perper = None
        try:
            inv = x.is_related_person.all()[0]
        except IndexError:
            inv = None
        try:
            main_person = inv.main_person.all()[0]
        except IndexError:
            main_person = None
        if main_person is not None:
            fam_rel_type, _ = SkosConcept.objects.get_or_create(pref_label="{}".format(ents[0]))
            fam_rel_type.collection.add(fam_scheme)
            perper, _ = PersonPerson.objects.get_or_create(
                source=x,
                target=main_person,
                rel_type=fam_rel_type
            )
            perper.save()
    else:
        pass

In [ ]:
# PersonPerson.objects.all().count()

In [ ]:
# PersonPerson.objects.all().delete()